In [28]:
import numpy as np
import pandas as pd
from sklearn.covariance import LedoitWolf
from scipy.optimize import minimize
import math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples 
from sklearn.neighbors import KernelDensity

In [75]:
#import data
Test_assets=pd.read_csv('49_Industry_Portfolios.CSV')
FF3factors=pd.read_csv('F-F_Research_Data_Factors.CSV')
#assign nas to missing values
Test_assets[Test_assets==-99.99]=math.nan
Test_assets[Test_assets==-999]=math.nan
#drop all nas, which makes sample period from 197001 to now
Test_assets=Test_assets.dropna()
Test_assets=Test_assets[Test_assets['date']>=197001]
FF3factors=FF3factors[FF3factors['date']>=198001]
#make returns into decimals
Test_assets.iloc[:,1:]=Test_assets.iloc[:,1:]/100
FF3factors.iloc[:,1:]=FF3factors.iloc[:,1:]/100

In [30]:
#functions to return mean vector and cov mat optional to perform Ledoit-Wolf shrinkage 
def getSampleCoefs(Obs,shrink=False):
    mu_vec=Obs.mean(axis=0)
    if shrink:
        Covmat=LedoitWolf().fit(Obs).covariance_
    else: Covmat=np.cov(Obs,rowvar=False)
    return np.array(mu_vec),Covmat
def fitKDE(obs,bWidth=.25,kernel='gaussian',x=None):
# Fit kernel to a series of obs, and derive the prob of obs
# x is the array of values on which the fit KDE will be evaluated 
    if len(obs.shape)==1:obs=obs.reshape(-1,1) 
    kde=KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs) 
    if x is None:x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1:x=x.reshape(-1,1) 
    logProb=kde.score_samples(x) # log(density) 
    pdf=pd.Series(np.exp(logProb),index=x.flatten())
    return pdf

def mpPDF(var,q,pts):
    # Marcenko-Pastur pdf
    # q=T/N
    eMin,eMax=var*(1-(1./q)**.5)**2,var*(1+(1./q)**.5)**2 
    eVal=np.linspace(eMin,eMax,pts) 
    pdf=q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5 
    pdf=pd.Series(pdf,index=eVal)
    return pdf
def errPDFs(var,eVal,q,bWidth,pts=1000):
    # Fit error
    pdf0=mpPDF(var,q,pts) # theoretical pdf 
    pdf1=fitKDE(eVal,bWidth,x=pdf0.index.values) # empirical pdf 
    sse=np.sum((pdf1-pdf0)**2)
    return sse
def findMaxEval(eVal,q,bWidth):
    # Find max random eVal by fitting Marcenko's dist to the empirical one
    out=minimize(lambda *x:errPDFs(*x),.5,args=(eVal,q,bWidth), bounds=((1E-5,1-1E-5),))
    if out['success']:var=out['x'][0] 
    else:var=1 
    eMax=var*(1+(1./q)**.5)**2 
    return eMax,var

In [31]:
#function to transform from cov mat to corr mat and vice versa
def CovtoCorr(cov):
    std=np.sqrt(np.diag(cov))
    corr=cov/np.outer(std,std) 
    corr[corr<-1],corr[corr>1]=-1,1 
    return corr
def Corr2Cov(corr,std):
    cov=corr*np.outer(std,std)
    return cov
#function to do diagnization 
def Geteigns(Xmat):
    # Get eVal,eVecs
    eVals,eVecs=np.linalg.eigh(Xmat)
    indices=eVals.argsort()[::-1] # index for sorting eVal desc 
    eVals,eVecs=eVals[indices],eVecs[:,indices] 
    #make the eigenvalues a diagnal mat
    eVals=np.diagflat(eVals)
    return eVals,eVecs
#function to shrink the Corr mat further by avgering small eignvalues
def deNoiseCov(cov0,n_pcs=None,q=None,bWidth=None):
    #get eigenvals,eigenvectors of corr
    corr0=CovtoCorr(cov0)
    eVal0,eVec0=Geteigns(corr0) 
    eVal1=eVal0.diagonal().copy()
    if n_pcs==None:
        #find eMax0
        eMax0,var0=findMaxEval(eVal=eVal1,q=q,bWidth=bWidth)
        #find n_pcs
        n_pcs=len(eVal1)-eVal1[::-1].searchsorted(eMax0)
    #average all the small eigenvalues
    eVal1[n_pcs:]=np.mean(eVal1[n_pcs:])
    eVal1=np.diag(eVal1)
    #compute shrinked corr mat
    corr1=np.dot(eVec0,eVal1).dot(eVec0.T)
    #get back to cov mat
    cov1=Corr2Cov(corr1,np.diag(cov0)**.5)
    return cov1
#function to estimate grand shrinked cov mat
def EstdeNoisedCoeff(Obs,n_pcs=None,q=None,bWidth=None,shrink=False):
    mu0,Cov0=getSampleCoefs(Obs=Obs,shrink=shrink)
    #regularize Cov mat
    Cov1=deNoiseCov(cov0=Cov0,n_pcs=n_pcs,q=q,bWidth=bWidth)
    return mu0,Cov1

In [32]:
#functions to do k-means
def clusterKMeansBase(corr0,maxNumClusters=None,n_init=10): 
    dist,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series() # define distance matrix 
    if maxNumClusters is None:maxNumClusters=round(corr0.shape[0]/2) 
    for init in range(n_init):
        for i in range(2,maxNumClusters+1): # find optimal num clusters 
            kmeans_=KMeans(n_clusters=i,n_jobs=1,n_init=1) 
            kmeans_=kmeans_.fit(dist) 
            silh_=silhouette_samples(dist,kmeans_.labels_) 
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]: 
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns 
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() for \
    i in np.unique(kmeans.labels_)} # cluster members 
    silh=pd.Series(silh,index=dist.index)
    return corr1,clstrs,silh

#function to do CVO
def optPort(cov,mu=None):
    inv=np.linalg.inv(cov) 
    ones=np.ones(shape=(inv.shape[0],1)) 
    if mu is None:mu=ones 
    #find the MVE port
    w=np.dot(inv,mu) 
    #make sure weights sum up to one 
    w/=np.dot(ones.T,w)
    return w
#function to do nco
def optPort_nco(cov,mu=None,maxNumClusters=None):
    cov=pd.DataFrame(cov)
    if mu is not None:mu=pd.Series(mu)
    corr1=CovtoCorr(cov) 
    corr1,clstrs,_=clusterKMeansBase(corr1,maxNumClusters,n_init=10) 
    wIntra=pd.DataFrame(0,index=cov.index,columns=clstrs.keys())
    for i in clstrs:
        cov_=cov.loc[clstrs[i],clstrs[i]].values
        mu_=(None if mu is None else mu.iloc[clstrs[i]].values.reshape(-1,1)) 
        wIntra.loc[clstrs[i],i]=optPort(cov_,mu_).flatten()
    cov_=wIntra.T.dot(np.dot(cov,wIntra)) # reduce covariance matrix 
    mu_=(None if mu is None else wIntra.T.dot(mu)) 
    wInter=pd.Series(optPort(cov_,mu_).flatten(),index=cov_.index) 
    nco=wIntra.mul(wInter,axis=1).sum(axis=1).values.reshape(-1,1)
    return nco

In [33]:
#function to return weights given obs,
#function to compute Port return given asset returns and weights
def compute_Port_ret(Obs,assetRets,n_pcs=None,shrink=False,Method='CVO',Benchmark=False,q=None,bWidth=None):
    if Benchmark:
        mu,Cov=getSampleCoefs(Obs=Obs)
        weights=optPort(cov=Cov,mu=mu)
    else:
        mu,Cov=EstdeNoisedCoeff(Obs=Obs,n_pcs=n_pcs,shrink=shrink,q=q,bWidth=bWidth)
        if (Method=='CVO'):
            weights=optPort(cov=Cov,mu=None)
        elif(Method=='NCO'):
            weights=optPort_nco(cov=Cov,mu=None)
    Port_Ret=np.dot(weights.T,assetRets)
    return Port_Ret

In [34]:
#initalization 
Port_rets=pd.DataFrame(data=np.zeros(shape=(Test_assets.iloc[120:,:].shape[0],4),),
                       index=Test_assets.iloc[120:,:].loc[:,'date'],
                       columns=['VWMkt','CVOnoShrink','CVOshrink','NCOshrink'])

In [36]:
#loop through Testset to do the back test monthly rebalance
for t in range(0,Port_rets.shape[0]):
    #we set a rolling window of 10 years to estimate mean and Cov
    Trainset=Test_assets.iloc[t:(120+t),1:]
    Testset=Test_assets.iloc[(120+t),1:]
    Port_rets.iloc[t,1]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Benchmark=True)
    Port_rets.iloc[t,2]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='CVO',q=120/49,bWidth=0.25)
    Port_rets.iloc[t,3]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='NCO',q=120/49,bWidth=0.25)
    print('Complete one run!')

Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete o

Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!


In [39]:
#get market excess return
Port_rets['VWMkt']=FF3factors['Mkt-RF'].values
#compute excess returns
Port_rets.iloc[:,1:]=np.array(Port_rets.iloc[:,1:])-np.repeat(FF3factors['RF'].values,3).reshape(484,3)
#get descrp stats
Des_Stats=Port_rets.describe()
#annulize results 
Des_Stats.loc['mean',]=Des_Stats.loc['mean',]*12
Des_Stats.loc['std',]=Des_Stats.loc['std',]*np.sqrt(12)
Des_Stats.loc['SharpeRatio',]=Des_Stats.loc['mean',]/Des_Stats.loc['std',]

In [41]:
Des_Stats

,VWMkt,CVOnoShrink,CVOshrink,NCOshrink
count,484.000000,484.000000,484.000000,484.000000
mean,0.078853,0.099586,0.074494,0.070874
std,0.155345,0.372926,0.126937,0.123519
min,-0.232400,-0.626546,-0.139607,-0.153751
25%,-0.019450,-0.049168,-0.015953,-0.015238
50%,0.011050,0.012943,0.007139,0.006625
75%,0.034600,0.066636,0.029264,0.029032
max,0.136500,0.388538,0.151024,0.138838
SharpeRatio,0.507599,0.267041,0.586856,0.573788


In [42]:
Port_rets.to_csv('MinVarPortRet_pdffit_monthly.csv')
Des_Stats.to_csv('DescripStats_pdffit_monthly.csv')

In [127]:
#initalization for quarterly rebalance
Port_rets=pd.DataFrame(data=np.zeros(shape=(Test_assets.iloc[[t for t in range(120,484,3)],:].shape[0],4)),
                       index=Test_assets.iloc[[t for t in range(120,484,3)],:].loc[:,'date'],
                       columns=['VWMkt','CVOnoShrink','CVOshrink','NCOshrink'])


In [128]:
#for quarterly rebalance
#loop through Testset to do the back test monthly rebalance
for t in range(0,round(Port_rets.shape[0])):
    #we set a rolling window of 10 years to estimate mean and Cov
    Trainset=Test_assets.iloc[3*t:(120+3*t),1:]
    Testset=Test_assets.iloc[(120+3*t):(123+3*t),1:]
    #compute Test set cum return over the period
    Testset=(np.cumprod(Testset+1)-1).iloc[2,:]
    Port_rets.iloc[t,1]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Benchmark=True)
    Port_rets.iloc[t,2]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='CVO',q=120/49,bWidth=0.25)
    Port_rets.iloc[t,3]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='NCO',q=120/49,bWidth=0.25)
    print('Complete one run!')

Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete o

In [131]:
#compute quarterly FF3 factors
FF3factors_quarter=(np.exp(np.log(FF3factors.iloc[:,1:]+1).rolling(3).sum())-1).iloc[[t for t in range(120,484,3)],:]
#get market excess return
Port_rets['VWMkt']=FF3factors_quarter['Mkt-RF'].values
#compute excess returns
Port_rets.iloc[:,1:]=np.array(Port_rets.iloc[:,1:])-np.repeat(FF3factors_quarter['RF'].values,3).reshape(Port_rets.shape[0],3)
#get descrp stats
Des_Stats=Port_rets.describe()
#annulize results 
Des_Stats.loc['mean',]=Des_Stats.loc['mean',]*4
Des_Stats.loc['std',]=Des_Stats.loc['std',]*np.sqrt(4)
Des_Stats.loc['SharpeRatio',]=Des_Stats.loc['mean',]/Des_Stats.loc['std',]

In [132]:
Des_Stats

,VWMkt,CVOnoShrink,CVOshrink,NCOshrink
count,122.000000,122.000000,122.000000,122.000000
mean,0.075682,0.098182,0.080898,0.075176
std,0.138230,0.411853,0.127855,0.124052
min,-0.237286,-0.603889,-0.183480,-0.207399
25%,-0.023050,-0.093056,-0.020628,-0.022342
50%,0.026492,0.001046,0.026685,0.025063
75%,0.067408,0.143326,0.059765,0.058048
max,0.192612,0.633448,0.153462,0.158187
SharpeRatio,0.547508,0.238390,0.632730,0.606007


In [133]:
Port_rets.to_csv('MinVarPortRet_pdffit_quarterly.csv')
Des_Stats.to_csv('DescripStats_pdffit_quarterly.csv')

In [134]:
#initalization for quarterly rebalance
Port_rets=pd.DataFrame(data=np.zeros(shape=(Test_assets.iloc[[t for t in range(120,484,12)],:].shape[0],4)),
                       index=Test_assets.iloc[[t for t in range(120,484,12)],:].loc[:,'date'],
                       columns=['VWMkt','CVOnoShrink','CVOshrink','NCOshrink'])

In [136]:
#for annual rebalance
#loop through Testset to do the back test annual rebalance
for t in range(0,round(Port_rets.shape[0])):
    #we set a rolling window of 10 years to estimate mean and Cov
    Trainset=Test_assets.iloc[12*t:(120+12*t),1:]
    Testset=Test_assets.iloc[(120+12*t):(132+12*t),1:]
    #compute Test set cum return over the period
    Testset=(np.cumprod(Testset+1)-1).iloc[11,:]
    Port_rets.iloc[t,1]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Benchmark=True)
    Port_rets.iloc[t,2]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='CVO',q=120/49,bWidth=0.25)
    Port_rets.iloc[t,3]=compute_Port_ret(Obs=Trainset,assetRets=Testset,Method='NCO',q=120/49,bWidth=0.25)
    print('Complete one run!')

Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!
Complete one run!


In [137]:
#compute annual FF3 factors
FF3factors_annual=(np.exp(np.log(FF3factors.iloc[:,1:]+1).rolling(12).sum())-1).iloc[[t for t in range(120,484,12)],:]
#get market excess return
Port_rets['VWMkt']=FF3factors_annual['Mkt-RF'].values
#compute excess returns
Port_rets.iloc[:,1:]=np.array(Port_rets.iloc[:,1:])-np.repeat(FF3factors_annual['RF'].values,3).reshape(Port_rets.shape[0],3)
#get descrp stats
Des_Stats=Port_rets.describe()
#annulize results 
Des_Stats.loc['mean',]=Des_Stats.loc['mean',]*1
Des_Stats.loc['std',]=Des_Stats.loc['std',]*np.sqrt(1)
Des_Stats.loc['SharpeRatio',]=Des_Stats.loc['mean',]/Des_Stats.loc['std',]

In [138]:
Des_Stats

,VWMkt,CVOnoShrink,CVOshrink,NCOshrink
count,31.000000,31.000000,31.000000,31.000000
mean,0.087982,0.077138,0.096850,0.093981
std,0.171851,0.432084,0.157101,0.149098
min,-0.389455,-0.712475,-0.227930,-0.259435
25%,-0.019486,-0.269245,-0.041863,-0.012103
50%,0.100707,0.057897,0.120644,0.108152
75%,0.219483,0.323559,0.179467,0.186254
max,0.367918,1.241444,0.377005,0.340268
SharpeRatio,0.511967,0.178526,0.616481,0.630329


In [139]:
Port_rets.to_csv('MinVarPortRet_pdffit_annual.csv')
Des_Stats.to_csv('DescripStats_pdffit_annual.csv')